In [59]:
import numpy as np
import warnings
import inspect
from euclid import *
# import carla

from collections.abc import MutableMapping
from cvxopt import matrix
import sys
import os
from bezier_path import Bezier

sys.path.append(os.path.dirname(os.path.abspath('')) +
                "../")

try:
    from cbf.cbf import KBM_VC_CBF2D
    from cbf.geometry import Rotation, Transform
    from cbf.obstacles import *
except:
    raise


In [30]:
class BBox():
    def __init__(self, wx, wy, l, b, h, theta):
        self.wx = wx
        self.wy = wy
        self.l = l
        self.b = b
        self.h = h
        self.theta = theta

class Ellipse2D():
    def __init__(self, a, b, buffer, cx=0, cy=0, theta=0):
        self.cx = cx
        self.cy = cy
        self.theta = theta
        self.a = a
        self.b = b
        self.buffer = buffer
        self.BUFFER_FLAG = False
    
    def applyBuffer(self):
        if not self.BUFFER_FLAG:
            self.a = self.a + self.buffer
            self.b = self.b + self.buffer
            self.BUFFER_FLAG = True
        else:
            print("Warning: Buffer already applied. Call Ignored.")
        
    def removeBuffer(self):
        if self.BUFFER_FLAG:
            self.a = self.a - self.buffer
            self.b = self.b - self.buffer
            self.BUFFER_FLAG = False
        else:
            print("Warning: Buffer already removed. Call Ignored.")
            
    deBuff = applyBuffer
    
    def update(self, cx, cy, theta, a, b, buffer):
        self.cx = cx
        self.cy = cy
        self.theta = theta
        self.a = a
        self.b = b
        self.buffer = buffer
    
    @classmethod
    def byBBox(cls, bBox, buffer):
        try:
            if not isinstance(bBox, BBox):
                raise ValueError("Expected an object of type BBox.")
            a = bBox.l/2
            b = bBox.b/2
            buffer = buffer
            cx = bBox.wx
            cy = bBox.wy
            theta = bBox.theta
        except ValueError as e:
            print(e)
        return cls(a, b, buffer, cx, cy, theta)     
    
    
        

In [31]:
bbox = BBox(2, 2, 3, 2, 2, np.pi/4)
ellipse1 = Ellipse2D(4, 2, 0.5, 3, 3, 0)

In [32]:
ellipse1.applyBuffer()
print(ellipse1.a)

4.5


In [33]:
bbx_ellipse = Ellipse2D.byBBox(bbox, 0.5)

In [34]:
bbx_ellipse.applyBuffer()
print(bbx_ellipse.a)

2.0


In [35]:
bbx_err_ellipse = Ellipse2D.byBBox(bbx_ellipse, 0.5)

Expected an object of type BBox.


UnboundLocalError: local variable 'a' referenced before assignment

In [36]:
c = "yoman"

In [37]:
warnings.warn("You have been warned")

/tmp/ipykernel_13068/2574046525.py:1: UserWarning: You have been warned
  warnings.warn("You have been warned")


In [38]:
bbx_ellipse.deBuff()

In [39]:
from collections.abc import MutableMapping


class TwoWayDict(MutableMapping):
    def __init__(self, data=()):
        self.mapping = {}
        self.update(data)
    def __getitem__(self, key):
        return self.mapping[key]
    def __delitem__(self, key):
        value = self[key]
        del self.mapping[key]
        self.pop(value, None)
    def __setitem__(self, key, value):
        if key in self:
            del self[self[key]]
        if value in self:
            del self[value]
        self.mapping[key] = value
        self.mapping[value] = key
    def __iter__(self):
        return iter(self.mapping)
    def __len__(self):
        return len(self.mapping)
    def __repr__(self):
        return f"{type(self).__name__}({self.mapping})"


In [40]:
test = TwoWayDict({1:2, 3:4})

In [41]:
test

TwoWayDict({1: 2, 2: 1, 3: 4, 4: 3})

In [42]:
del test[2]

In [43]:
test

TwoWayDict({3: 4, 4: 3})

In [44]:
type(test.super())

AttributeError: 'TwoWayDict' object has no attribute 'super'

In [45]:
test.__class__.__mro__

(__main__.TwoWayDict,
 collections.abc.MutableMapping,
 collections.abc.Mapping,
 collections.abc.Collection,
 collections.abc.Sized,
 collections.abc.Iterable,
 collections.abc.Container,
 object)

In [46]:
#!/bin/python3
"""

The Obstacle classes containing the neccessary gradients and hessian functions for
seamless integration with optimal solvers, includes several utility objects like 
the obstacle list for use in real time simulation.

author: Neelaksh Singh

"""

# Identity Objects
DICT_EMPTY_UPDATE = ()

# Object Selectors for utility
ELLIPSE2D = 0

class Obstacle2DBase():
    """
    The base class each 2D obstacle class will inherit from. Created to enforce specific
    validation checks in the obstacle list objects and creating the neccessary interface
    for all 2D obstacle CBF classes.
    """
    def __init__(self):
        pass

    def evaluate(self, p):
        if not isinstance(p, Vector2):
            raise ValueError("Expected an object of type euclid.Vector2 for arg p, but got " + type(p).__name__ + ".")

    def gradient(self, p):
        if not isinstance(p, Vector2):
            raise ValueError("Expected an object of type euclid.Vector2 for arg p, but got " + type(p).__name__ + ".")
        return matrix(0.0, (3,1))

    def f(self, p):
        if not isinstance(p, Vector2):
            raise ValueError("Expected an object of type euclid.Vector2 for arg p, but got " + type(p).__name__ + ".")
        return 0
    
    def dx(self, p):
        if not isinstance(p, Vector2):
            raise ValueError("Expected an object of type euclid.Vector2 for arg p, but got " + type(p).__name__ + ".")
        return 0
    
    def dy(self, p):
        if not isinstance(p, Vector2):
            raise ValueError("Expected an object of type euclid.Vector2 for arg p, but got " + type(p).__name__ + ".")
        return 0

    def dtheta(self, p):
        if not isinstance(p, Vector2):
            raise ValueError("Expected an object of type euclid.Vector2 for arg p, but got " + type(p).__name__ + ".")
        return 0

    def update(self):
        pass

    def updateCoords(self, xy):
        if not isinstance(xy, Vector2):
            raise ValueError("Expected an object of type euclid.Vector2 for arg p, but got " + type(xy).__name__ + ".")
        pass

    def updateOrientation(self):
        pass

class Ellipse2D(Obstacle2DBase):
    def __init__(self, a, b, center = Vector2(0, 0), theta=0, buffer=0):
        """
        Generates the 2D Ellipse obstacle representation for use in control barrier functions.
        Exposes the required functionality for direct usage in CBF as a barrier constraint.

        """
        if not isinstance(center, Vector2):
            raise ValueError("Expected an object of type euclid.Vector2 for arg center, but got " + type(center).__name__ + ".")
        self.center = center
        self.theta = theta
        self.a = a + buffer
        self.b = b + buffer
        self.buffer = buffer
        self.BUFFER_FLAG = True
    
    def applyBuffer(self):
        if not self.BUFFER_FLAG:
            self.a = self.a + self.buffer
            self.b = self.b + self.buffer
            self.BUFFER_FLAG = True
        else:
            warnings.warn("Warning: Buffer already applied. Call Ignored.")
        
    def removeBuffer(self):
        if self.BUFFER_FLAG:
            self.a = self.a - self.buffer
            self.b = self.b - self.buffer
            self.BUFFER_FLAG = False
        else:
            warnings.warn("Warning: Buffer already removed. Call Ignored.")
    
    def evaluate(self, p):
        """
        Evaluate the value of the ellipse at a given point.
        """
        super().evaluate(p)
        dx = p.x - self.center.x
        dy = p.y - self.center.y
        ct = np.cos(self.theta)
        st = np.sin(self.theta)

        eval = ( ( dx * ct + dy * st )/self.a )**2 + ( ( -dx * st + dy * ct )/self.b )**2 - 1
        return eval

    def gradient(self, p):
        super().gradient(p)
        return matrix([self.dx(p), self.dy(p), self.dtheta(p)])

    # f = evaluate
        
    def f(self, p):
        """
        Alias of the evaluate function, semantically significant for cvxopt.
        """
        return self.evaluate(p)
    
    def dx(self, p):
        super().dx(p)
        xd = p.x - self.center.x
        yd = p.y - self.center.y
        ct = np.cos(self.theta)
        st = np.sin(self.theta)

        dx_ = (2 * ct/(self.a**2)) * ( xd * ct + yd * st ) + (-2 * st/(self.b**2)) * ( -xd * st + yd * ct )
        return dx_
    
    def dy(self, p):
        super().dy(p)
        xd = p.x - self.center.x
        yd = p.y - self.center.y
        ct = np.cos(self.theta)
        st = np.sin(self.theta)

        dy_ = (2 * st/(self.a**2)) * ( xd * ct + yd * st ) + (2 * ct/(self.b**2)) * ( -xd * st + yd * ct )
        return dy_
    
    def update(self, a=None, b=None, center=None, theta=None, buffer=None):
        if a is not None:
            self.a = a
        if b is not None:
            self.b = b
        if center is not None:
            if not isinstance(center, Vector2):
                raise ValueError("Expected an object of type euclid.Vector2 for arg center.")
            self.center = center
        if theta is not None:
            self.theta = theta
        if buffer is not None:
            if self.BUFFER_FLAG:
                self.a = self.a - self.buffer + buffer
                self.b = self.b - self.buffer + buffer
                self.buffer = buffer
            else:
                self.buffer = buffer
    
    def updateCoords(self, xy):
        super().updateCoords(xy)
        self.center = xy

    def updateOrientation(self, yaw):
        self.theta = yaw

    def updateByBoundingBox(self, BBox):
        a = BBox.extent.x
        b = BBox.extent.y
        center = Vector2(BBox.location.x, BBox.location.y)
        theta = BBox.rotation.yaw
        self.update(a=a, b=b, center=center, theta=theta)

    def dtheta(self, p):
        """
        Despite being zero. This function is still created for the sake of completeness w.r.t API.
        """
        return super().dtheta(p)

    def __repr__(self):
        return f"{type(self).__name__}(a = {self.a},\n b = {self.b},\n center = {self.center},\n theta = {self.theta},\n buffer = {self.buffer}, \n buffer applied: {self.BUFFER_FLAG} )"
    
    @classmethod
    def fromBoundingBox(cls, BBox, buffer = 0.5):
        if not isinstance(BBox, carla.BoundingBox):
            raise ValueError("Expected an object of type carla.BoundingBox as an input to fromCarlaBoundingBox() method, but got ", type(BBox).__name__)
        
        a = BBox.extent.x
        b = BBox.extent.y
        center = Vector2(BBox.location.x, BBox.location.y)
        theta = BBox.rotation.yaw
        return cls(a, b, center, theta, buffer)

In [68]:
ellipse2d1 = Ellipse2D(2,3)
ellipse2d2 = Ellipse2D(3,4)

In [69]:
obs2dlist = ObstacleList2D({"id1":ellipse2d1, "id2":ellipse2d2})

In [70]:
obs2dlist

ObstacleList2D({'id1': Ellipse2D(a = 2, b = 3, center = Vector2(0.00, 0.00), theta = 0, buffer = 0, buffer_applied: True )
, 'id2': Ellipse2D(a = 3, b = 4, center = Vector2(0.00, 0.00), theta = 0, buffer = 0, buffer_applied: True )
})

In [71]:
p = Vector2(2,0)

In [72]:
print(obs2dlist.f(p))

[ 0.00e+00]
[-5.56e-01]



In [73]:
ellipse2d3 = Ellipse2D(5,5)

In [74]:
obs2dlist.update(id3 = ellipse2d3)

In [75]:
obs2dlist

ObstacleList2D({'id1': Ellipse2D(a = 2, b = 3, center = Vector2(0.00, 0.00), theta = 0, buffer = 0, buffer_applied: True )
, 'id2': Ellipse2D(a = 3, b = 4, center = Vector2(0.00, 0.00), theta = 0, buffer = 0, buffer_applied: True )
, 'id3': Ellipse2D(a = 5, b = 5, center = Vector2(0.00, 0.00), theta = 0, buffer = 0, buffer_applied: True )
})

In [76]:
obs2dlist['id4'] = ellipse2d3

In [77]:
obs2dlist

ObstacleList2D({'id1': Ellipse2D(a = 2, b = 3, center = Vector2(0.00, 0.00), theta = 0, buffer = 0, buffer_applied: True )
, 'id2': Ellipse2D(a = 3, b = 4, center = Vector2(0.00, 0.00), theta = 0, buffer = 0, buffer_applied: True )
, 'id3': Ellipse2D(a = 5, b = 5, center = Vector2(0.00, 0.00), theta = 0, buffer = 0, buffer_applied: True )
, 'id4': Ellipse2D(a = 5, b = 5, center = Vector2(0.00, 0.00), theta = 0, buffer = 0, buffer_applied: True )
})

In [78]:
obs2dlist['id4'].update(a=5, b=6)

In [79]:
obs2dlist

ObstacleList2D({'id1': Ellipse2D(a = 2, b = 3, center = Vector2(0.00, 0.00), theta = 0, buffer = 0, buffer_applied: True )
, 'id2': Ellipse2D(a = 3, b = 4, center = Vector2(0.00, 0.00), theta = 0, buffer = 0, buffer_applied: True )
, 'id3': Ellipse2D(a = 5, b = 6, center = Vector2(0.00, 0.00), theta = 0, buffer = 0, buffer_applied: True )
, 'id4': Ellipse2D(a = 5, b = 6, center = Vector2(0.00, 0.00), theta = 0, buffer = 0, buffer_applied: True )
})

In [81]:
"""Defining the trajectory"""
start_x = 5.3  # [m]
start_y = 46.2  # [m]
start_yaw = np.radians(-90.0)  # [rad]

end_x = 28.7  # [m]
end_y = 7.0  # [m]
end_yaw = np.radians(0.0)  # [rad]
offset = 3.0
resolution = 100
velocity = 5

bezier = Bezier(start_x, start_y, start_yaw, end_x, end_y, end_yaw, offset, resolution=100)
curve = bezier.get_trajectory(velocity= velocity)

straight1 = []
for t in np.linspace(66.2, 46.2, resolution):
    straight1.append((5.3, t, -math.pi/2, velocity))

straight2 = []
for t in np.linspace(28.7, 54.8, resolution):
    straight2.append((t, 7.0, -math.pi/2, velocity))

trajectory = straight1 + curve + straight2

In [82]:
trajectory

[(5.3, 66.2, -1.5707963267948966, 5),
 (5.3, 65.9979797979798, -1.5707963267948966, 5),
 (5.3, 65.7959595959596, -1.5707963267948966, 5),
 (5.3, 65.5939393939394, -1.5707963267948966, 5),
 (5.3, 65.3919191919192, -1.5707963267948966, 5),
 (5.3, 65.18989898989899, -1.5707963267948966, 5),
 (5.3, 64.98787878787878, -1.5707963267948966, 5),
 (5.3, 64.78585858585859, -1.5707963267948966, 5),
 (5.3, 64.58383838383838, -1.5707963267948966, 5),
 (5.3, 64.38181818181819, -1.5707963267948966, 5),
 (5.3, 64.17979797979798, -1.5707963267948966, 5),
 (5.3, 63.97777777777778, -1.5707963267948966, 5),
 (5.3, 63.77575757575758, -1.5707963267948966, 5),
 (5.3, 63.57373737373737, -1.5707963267948966, 5),
 (5.3, 63.37171717171717, -1.5707963267948966, 5),
 (5.3, 63.16969696969697, -1.5707963267948966, 5),
 (5.3, 62.96767676767677, -1.5707963267948966, 5),
 (5.3, 62.76565656565657, -1.5707963267948966, 5),
 (5.3, 62.56363636363637, -1.5707963267948966, 5),
 (5.3, 62.36161616161616, -1.5707963267948966, 5